In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
SCORE_DIR = "data/scores"

In [3]:
box_scores = os.listdir(SCORE_DIR)    # Get box scores from score directory

In [4]:
box_scores = [os.path.join(SCORE_DIR, f) for f in box_scores if f.endswith(".html")]    # Join score directory and file name filtering by ".html"

SyntaxError: invalid syntax (2301654723.py, line 1)

In [5]:
# Parse box scoes with BeautifulSoup
def parse_html(box_score):                                    # Parse html for clean up
    with open(box_score) as f:                                # Read box score file
        html = f.read()

    soup = BeautifulSoup(html)                                # Create BeautifulSoup instance to parse read file
    [s.decompose() for s in soup.select("tr.over_header")]    # Remove "tr.over_header" using select() and decompose()
    [s.decompose() for s in soup.select("tr.thead")]          # Remove "tr.thead" using select() and decompose()
    return soup                                               # Return a cleaned soup - decompose() runs in place, so we don't need to assign above methods to anything

In [ ]:
# Read line score table with pandas
def read_line_score(soup):                                              # Read line score that indicates how many points were scored and who won/lost
    line_score = pd.read_html(str(soup), attrs={"id": "line_score"})    # Given html, use pd.read_html() to process table with id "line_score"
    cols = list(line)score.columns)                                     # Fix column names by convert column names from list to index
    cols[0] = "team"                                                    # First column value assigned to "team"
    cols [-1] = "total"                                                 # Second column value assigned to "total"
    line_score.columns = cols                                           # Assign modified cols to dataframe columns

    line_score = line_score[["team", "total"]]                          # Remove column names between "team" and "total" ("1", "2", "3", "4" because of overtime game errors) 
    return line_score

In [6]:
# Parse basic & advanced stats tables with pandas
def read_stats(soup, team, stat):
    df = pd.read_html(str(soup), attrs={"id": f"box-{team}-game-{stat}"}, index_col = 0)[0]     # Read stats varying by team, basic, and advanced
    df = df.apply(pd.to_numeric, errors="coerce")                                               # Convert columns to numeric columns to & replace strings with NaN
    return df

In [ ]:
box_score = box_scores[0]
soup = parse_html(box_score)
line_score = read_line_score(soup)
teams = list(line_score["team"])

summaries = []
for team in teams:
    basic = read_stats(soup, team, "basic")
    advanced = read_stats(soup, team, "advanced")

    total = pd.concat([basic.iloc[-1,:], advanced.iloc[-1,:]])
    totals = totals.index.str.lower()                                           # Takes all column names and converts to lower-case
      
    maxes = pd.concat([basic.iloc[:-1,:].max(), advanced.iloc[:-1, :].max()])
    maxes.index = maxes.index.str.lower() + "_max"

    summary = pd.concat([totals, maxes])